# Computational Theory Problems

## Problem 1: Binary Words and Operations

In [15]:
import numpy as np

# Configure numpy to work with 32-bit unsigned integers
UINT32 = np.uint32

def Parity(x, y, z):
    """
    Parity function: (x ⊕ y ⊕ z)
    
    Reference: SHS Standard (FIPS PUB 180-4), Section 4.1.1 (page 10)
    This section defines how Parity is used in the SHA-1 algorithm.
    The Parity function is specified as x XOR y XOR z.
    Used in: SHA-1 algorithm (rounds 20-39 and 60-79)
    
    Computes the bitwise XOR on each bit of three 32-bit words.
    Examples: returns 1 if an odd number of inputs are 1, else returns 0.
    x == 1 y == 1 z == 0 result == 0
    x == 1 y == 0 z == 1 result == 0
    x == 0 y == 1 z == 1 result == 0 
    x == 1 y == 1 z == 1 result == 1

    Parity = 0b0001 = 0b1
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result of x XOR y XOR z
    
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32(x ^ y ^ z)

In [16]:
# Example test for the Parity function
x, y, z = UINT32(0b1101), UINT32(0b1011), UINT32(0b0111)

print("x:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Parity:", bin(int(Parity(x, y, z)))) # Expected output: 0b1 (result of x XOR y XOR z)


x: 0b1101
y: 0b1011
z: 0b111
Parity: 0b1


In [17]:
def Ch(x, y, z):
    """
    Choose function: (x ∧ y) ⊕ (¬x ∧ z)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    This section defines how the Choose function is used in the SHA-256 algorithm.
    The Choose function selects bits from y and z based on the bits of x.
    Used in: SHA-256 compression function
    
    For each bit position:
    - If x bit is 1, choose the corresponding bit from y
    - If x bit is 0, choose the corresponding bit from z
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result where x "chooses" between y and z
    
    Example:
        x = 1100 (chooses y for first 2 bits, z for last 2 bits)
        y = 1010
        z = 0111
        --------
        Result = 1011 (takes 10 from y, 11 from z)
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32((x & y) ^ (~x & z))

In [18]:
# Example test for the Ch function
x, y, z = UINT32(0b1100), UINT32(0b1010), UINT32(0b0111)

print("\nx:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Ch:", bin(int(Ch(x, y, z)))) # Expected output: 0b1011


x: 0b1100
y: 0b1010
z: 0b111
Ch: 0b1011


In [19]:
def Maj(x, y, z):
    """
    Majority function: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    This section defines how the Majority function is used in the SHA-256 algorithm.
    The Majority function returns the majority bit for each bit position among x, y, and z
    Used in: SHA-256 compression function
    
    For each bit position:
    - Returns 1 if at least 2 of the 3 bits are 1
    - Returns 0 if at least 2 of the 3 bits are 0
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result representing the majority vote of each bit position
    
    Example:
        x = 1100
        y = 1010
        z = 1001
        --------
        Bit 0: 1,1,1 → majority is 1
        Bit 1: 1,0,0 → majority is 0
        Bit 2: 0,1,0 → majority is 0
        Bit 3: 0,0,1 → majority is 0
        Result = 1000
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32((x & y) ^ (x & z) ^ (y & z))

In [20]:
x, y, z = UINT32(0b1100), UINT32(0b1010), UINT32(0b1001)

print("\nx:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Maj:", bin(int(Maj(x, y, z)))) # Expected output: 0b1000


x: 0b1100
y: 0b1010
z: 0b1001
Maj: 0b1000


In [21]:
def ROTR(x, n, w=32):
    """
    Rotate right (circular right shift) operation.
    
    Reference: SHS Standard, Section 2.2.2 (page 5)
    This section defines the ROTR operation used in SHA algorithms.
    ROTR is used in various SHA functions to achieve bit diffusion.
    
    Args:
        x: w-bit word (uint32)
        n: number of positions to rotate (0 <= n < w)
        w: word size in bits (default: 32)
    
    Returns:
        Result of rotating x right by n positions
    """
    x = UINT32(x)
    n = int(n) % w
    return UINT32((x >> n) | (x << (w - n)))

In [22]:
def Sigma0(x):
    """
    Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Upper case Sigma 0 for SHA-256
    Used in: SHA-256 compression function (transforms working variable 'a')
    
    Takes the input x and:
    1. Rotates it right by 2 positions
    2. Rotates it right by 13 positions
    3. Rotates it right by 22 positions
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the Σ₀ transformation
    
    Example:
        If x = 0xABCD1234
        - ROTR²(x) = rotate right 2
        - ROTR¹³(x) = rotate right 13
        - ROTR²²(x) = rotate right 22
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

In [23]:
x = UINT32(0xABCD1234)

print("\nx:", hex(int(x)))
print("ROTR²(x):", hex(int(ROTR(x, 2))))
print("ROTR¹³(x):", hex(int(ROTR(x, 13))))
print("ROTR²²(x):", hex(int(ROTR(x, 22))))
print("Sigma0:", hex(int(Sigma0(x)))) # Expected output: 0x8f1ec84a


x: 0xabcd1234
ROTR²(x): 0x2af3448d
ROTR¹³(x): 0x91a55e68
ROTR²²(x): 0x3448d2af
Sigma0: 0x8f1ec84a


In [24]:
def Sigma1(x):
    """
    Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Upper case Sigma 1 for SHA-256
    Used in: SHA-256 compression function (transforms working variable 'e')
    
    
    
    
    Takes the input x and:
    1. Rotates it right by 6 positions
    2. Rotates it right by 11 positions
    3. Rotates it right by 25 positions
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the Σ₁ transformation
    
    Example:
        If x = 0x9A8B7C6D
        - ROTR⁶(x) = rotate right 6
        - ROTR¹¹(x) = rotate right 11
        - ROTR²⁵(x) = rotate right 25
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

In [25]:
# Example test for the Sigma1 function
x = UINT32(0x9A8B7C6D)

print("\nx:", hex(int(x)))
print("ROTR⁶(x):", hex(int(ROTR(x, 6))))
print("ROTR¹¹(x):", hex(int(ROTR(x, 11))))
print("ROTR²⁵(x):", hex(int(ROTR(x, 25))))
print("Sigma1:", hex(int(Sigma1(x)))) # Expected output: 0x7e674a53


x: 0x9a8b7c6d
ROTR⁶(x): 0xb66a2df1
ROTR¹¹(x): 0x8db3516f
ROTR²⁵(x): 0x45be36cd
Sigma1: 0x7e674a53


In [26]:
def SHR(x, n):
    """
    Right shift operation.
    
    Reference: SHS Standard, Section 2.2.2 (page 6)
    Definition: SHR_n(x) = x >> n
    
    Args:
        x: w-bit word (uint32)
        n: number of positions to shift (0 <= n < w)
    
    Returns:
        Result of shifting x right by n positions
    """
    return UINT32(UINT32(x) >> n)

In [27]:
def sigma0(x):
    """
    σ₀²⁵⁶(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Lower case sigma 0 for SHA-256

    
    Takes the input x and:
    1. Rotates it right by 7 positions
    2. Rotates it right by 18 positions
    3. SHIFTS it right by 3 positions
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the σ₀ transformation
    
    
    Example:
        If x = 0x12345678
        - ROTR⁷(x) = rotate right 7 (bits wrap)
        - ROTR¹⁸(x) = rotate right 18 (bits wrap)
        - SHR³(x) = shift right 3 (bits lost!)
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))


In [28]:
# Example test for the sigma0 function
x = UINT32(0x12345678)

print("\nx:", hex(int(x)))
print("ROTR⁷(x):", hex(int(ROTR(x, 7))))
print("ROTR¹⁸(x):", hex(int(ROTR(x, 18))))
print("SHR³(x):", hex(int(SHR(x, 3))))
print("sigma0:", hex(int(sigma0(x))))


x: 0x12345678
ROTR⁷(x): 0xf02468ac
ROTR¹⁸(x): 0x159e048d
SHR³(x): 0x2468acf
sigma0: 0xe7fce6ee


In [29]:
def sigma1(x):
    """
    σ₁²⁵⁶(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Lower case sigma 1 for SHA-256
    
    Takes the input x and:
    1. Rotates it right by 17 positions
    2. Rotates it right by 19 positions
    3. SHIFTS it right by 10 positions 
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the σ₁ transformation
    
    
    Example:
        If x = 0xFEDCBA98
        - ROTR¹⁷(x) = rotate right 17 (bits wrap)
        - ROTR¹⁹(x) = rotate right 19 (bits wrap)
        - SHR¹⁰(x) = shift right 10 (bits lost!)
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

In [ ]:
x = UINT32(0xFEDCBA98)

print("\nx:", hex(int(x)))
print("ROTR¹⁷(x):", hex(int(ROTR(x, 17))))
print("ROTR¹⁹(x):", hex(int(ROTR(x, 19))))
print("SHR¹⁰(x):", hex(int(SHR(x, 10))))
print("sigma1:", hex(int(sigma1(x)))) 


x: 0xfedcba98
ROTR¹⁷(x): 0x5d4c7f6e
ROTR¹⁹(x): 0x97531fdb
SHR¹⁰(x): 0x3fb72e
sigma1: 0xca20d79b


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords